In [1]:
import requests
from bs4 import BeautifulSoup as soup
import os, time
import json,re
import random

In [2]:
main_url = "https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;filter=advanced;orderby=matches;size=150;spanmin1=01+Jan+1980;spanval1=span;team=8;template=results;type=allround"

In [3]:
response = requests.get(
    url = main_url,
)

main_soup = soup(response.content, 'html.parser')

In [4]:
cricketers_list_raw = main_soup.find_all('tr', "data1")

In [5]:
cricketers_list = []
for i in range(0,100):
    for x in cricketers_list_raw[i].find_all("a", "data-link"):
        cricketers_list.append({"name": x.getText(), "url": x.get("href")})
    

In [6]:
# cricketers_list

In [7]:
base_url_player = "https://stats.espncricinfo.com/"

In [8]:
# Filter dictionary keeping necessary fields
def filter_dict(di, keys):
    if not isinstance(di, (dict, list)):
        return di
    if isinstance(di, list):
        return [filter_dict(value, keys) for value in di]
    return {key: filter_dict(value, keys) for key, value in di.items() if key in keys or isinstance(value, (dict, list))}


In [9]:
player_data_keys = ["id", "Full_Name", "Born", "Age", "Batting_Style", "Bowling_Style", "Playing_Role", "Education"]
def extract_player_data(soup, id):
    player_card = soup.find_all('div', "player_overview-grid")
    player_details_raw = player_card[0].find_all('div')
    player_details = {}
    player_details["id"] = id
    for detail in player_details_raw:
        if(detail.p.getText() == "Fielding Position"):
            continue
        player_details[detail.p.getText().replace(" ", "_")] = detail.h5.getText()
    if(not "Bowling_Style" in player_details):
        player_details["Bowling_Style"] = "N/A"
    if(not "Playing_Role" in player_details):
        player_details["Playing_Role"] = "N/A"
    if(not "Education" in player_details):
        player_details["Education"] = "N/A"
    # print(player_details["Full Name"])
    return filter_dict(player_details, player_data_keys)
    

In [10]:
def extract_odi_data(soup):
    standings_card_raw = soup.find_all("div", "overflow-hidden")
    tables_raw_check = standings_card_raw[2].find_all("h5")
    
    # Finding Batting and Bowling tables and extracting the table
    for i in range (2):
        if(tables_raw_check[i].getText()=="Batting & Fielding"):
            batting_table_raw = tables_raw_check[i].find_next("table", "table standings-widget-table text-center mb-0 border-bottom")
        else:
            bowling_table_raw = tables_raw_check[i].find_next("table", "table standings-widget-table text-center mb-0 border-bottom")

    player_values = {}
    # Batting Values extraction
    # Finding ODI row and extracting the row
    batting_table_rows_raw = batting_table_raw.find("tbody").find_all("tr")
    for row in batting_table_rows_raw:
        if (row.find("span").getText()=="ODI"):
            batting_table_odi_row = row
            break
        else:
            continue
    batting_values_raw = batting_table_odi_row.find_all("span")
    player_values["Matches"] = int(batting_values_raw[1].getText())
    player_values["Runs"] = int(batting_values_raw[4].getText())
    player_values["Average"] = float(batting_values_raw[6].getText())
    player_values["Half-centuries"] = int(batting_values_raw[10].getText())
    player_values["Centuries"] = int(batting_values_raw[9].getText())

    # Bowling values extraction
    # Finding ODI row and extracting the row
    bowling_table_rows_raw = bowling_table_raw.find("tbody").find_all("tr")
    for row in bowling_table_rows_raw:
        if (row.find("span").getText()=="ODI"):
            bowling_table_odi_row = row
            break
        else:
            continue
    bowling_values_raw = bowling_table_odi_row.find_all("span")
    player_values["Wickets"] = int(bowling_values_raw[5].getText()) if(bowling_values_raw[5].getText()!="-") else 0
    player_values["Economy"] = float(bowling_values_raw[9].getText()) if(bowling_values_raw[9].getText()!="-") else 0.0
    return player_values
    

In [11]:
# Main Function
cricketers = []
for i in range(0,100):
    # print(i)
    player_url = base_url_player + cricketers_list[i]["url"]
    response = requests.get(
    url = player_url,
    )
    player_soup = soup(response.content, 'html.parser')
    player_data = extract_player_data(player_soup, i) | extract_odi_data(player_soup)
    cricketers.append(player_data)

In [87]:
# print(cricketers)

In [57]:
import pandas as pd

my_dict = [ {'type' : 'type_1', 'prob' : 2, 'x_sum' : 3, 'y_sum' : 5},
 {'type' : 'type_2', 'prob' : 3, 'x_sum' : 8, 'y_sum' : 6}]

df = pd.DataFrame(cricketers_si)
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys',tablefmt='psql')
# print(pdtabulate(df))


In [23]:
# cric = []
# for cricketer in cricketers:
#     cric.append({k: cricketer[k] for k in cricketer.keys() & {'Batting Style', 'Bowling Style', 'Playing Role'}})

# s = set( val for dic in cric for val in dic.values())

# for x in s:                                
    # print (x)
# print (len(s)) 

In [24]:
# schools = []
# for cricketer in cricketers:
#     schools.append({k: cricketer[k] for k in cricketer.keys() & {'Education'}})

# s2 = set( val for dic in schools for val in dic.values())

# for x in s2:                                
#     print (x)
# print (len(s2)) 

In [12]:
si_batting_bowling_playing_role = {"Left arm fast medium":"",
"Slow left arm orthodox":"",
"Right arm medium fast": "දකුණත් මද වේග පන්දු යවන",
"Legbreak googly": "", 
"Legbreak": "පා දඟ පන්දු යවන",
"Left arm medium fast": "වමත් මද වේග පන්දු යවන",
"Top order batter":  "ඉදිරි පෙළ පිතිකරු",
"Right hand bat": "දකුණත් පිතිකරු",
"Right arm offbreak, Legbreak": "දකුණත් දඟ පන්දු හා පා දඟ පන්දු යවන",
"Batter":"පිතිකරු",
"Middle order batter": "මැද පෙළ පිතිකරු",
"Left hand bat": "වමත් පිතිකරු",
"Bowling allrounder": "පන්දු යවන තුන් ඉරියව් ක්‍රීඩක",
"Bowler":"පන්දු යවන්නා",
"Wicketkeeper batter": "කඩුලු රකින පිතිකරු",
"Batting allrounder": "පිතිකරණ තුන් ඉරියව් ක්‍රීඩක",
"Allrounder": "තුන් ඉරියව් ක්‍රීඩක",
"Right arm medium": "දකුණත් මද පන්දු යවන",
"Left arm medium, Slow left arm orthodox": "වමත් මද පන්දු යවන",
"Right arm offbreak": "දකුණත් දඟ පන්දු යවන",
"Right arm fast": "දකුණත් වේග පන්දු යවන",
"Opening batter": "ආරම්භක පිතිකරු",
"Right arm fast medium": ""}

In [13]:
si_schools = {
    "Royal College, Colombo": "රාජකීය විද්‍යාලය, කොළඹ",
"Ananda College": "ආනන්ද විද්‍යාලය",
"St. Joseph's College, Maradana": "සාන්ත ජෝසප් විද්‍යාලය, මරදාන",
"St. Joseph's College Maradana": "සාන්ත ජෝසප් විද්‍යාලය, මරදාන",
"Mahanama College, Colombo": "මහනාම විද්‍යාලය, කොළඹ",
"Kalutara Vidyalaya": "කළුතර විද්‍යාලය",
"St. Sebastian's College, Moratuwa": "සාන්ත සෙබස්තියන් විද්‍යාලය, මොරටුව",
"Prince of Wales College, Moratuwa": "වේල්ස් කුමර විද්‍යාලය, මොරටුව",
"Sri Sumangala College, Panadura":"ශ්‍රී සුමංගල විද්‍යාලය, පානදුර",
"St. Annes College, Kurunegala": "සාන්ත ආනා විද්‍යාලය, කුරුණෑගල",
"S' Thomas' College, Colombo": "සාන්ත තෝමස් විද්‍යාලය, කොළඹ",
"Rahula College": "රාහුල විද්‍යාලය",
"Ananda College, Colombo": "ආනන්ද විද්‍යාලය, කොළඹ",
"St. Peters College - Negombo, Maristella College": "සාන්ත පීතර විද්‍යාලය, මාරිස් ස්ටෙලා විද්‍යාලය, මීගමුව",
"Richmond College, Galle": "රිච්මන්ඩ් විද්‍යාලය, ගාල්ල",
"St. Peter's College": "සාන්ත පීතර විද්‍යාලය",
"St. Peters College, Colombo": "සාන්ත පීතර විද්‍යාලය",
"Maliyadewa College, Kurunegala": "මලියදේව විද්‍යාලය, කුරුණෑගල",
"St. Joseph's College, Colombo": "සාන්ත ජෝසප් විද්‍යාලය, කොළඹ",
"Mahanama Vidyalaya, Panadura": "මහනාම විද්‍යාලය, පානදුර",
"Nalanda College, Colombo": "නාලන්ද විද්‍යාලය, කොළඹ",
"De Mazenod College, Kandana": "ද මස්නෝද් විද්‍යාලය, කඳාන",
"St Anthony's College, Kandy": "සාන්ත අන්තෝනි විද්‍යාලය, මහනුවර",
"Debarawewa Central Hambantota, Mahanama College Colombo, Richmond College, Galle": "දෙබරවැව මධ්‍ය මහා විද්‍යාලය හම්බන්තොට, මහනාම විද්‍යාලය කොළඹ, රිච්මන්ඩ් විද්‍යාලය ගාල්ල",
"Rewatha College, Balapitiya": "රේවත විද්‍යාලය, බලපිටිය",
"Rahula College, Matara":  "රාහුල විද්‍යාලය, මාතර",
"Maris Stella College": "මාරිස් ස්ටෙලා විද්‍යාලය",
"Trinity College, Kandy": "ත්‍රිත්ව විද්‍යාලය, මහනුවර",
"St. Aloysius' College, Galle": "සාන්ත ඇලෝසියස් විද්‍යාලය, ගාල්ල",
"St. Mary's College, Chilaw": "සාන්ත මරියා විද්‍යාලය, හලාවත",
"Vidhyaloka Maha Vidyalaya, Katana": "විද්‍යාලෝක මහා විද්‍යාලය, කටාන",
"Ananda Sastralaya, Kotte": "ආනන්ද ශාස්ත්‍රාලය, කෝට්ටේ",
"St. John's College, Panadura": "සාන්ත ජෝන් විද්‍යාලය, පානදුර",
}


In [14]:
cricketers_si = []
for i in range(0,100):
    cricketers_si.append({key: si_batting_bowling_playing_role.get(value, value) for key, value in cricketers[i].items()})
    cricketers_si[i]["Player_Bio"] = "ක්රීඩකයාගේ භූමිකාව: "+ cricketers_si[i]["Playing_Role"] + ". ඔහුගේ පිතිකරණ රටාව: "+ cricketers_si[i]["Batting_Style"] + ". ඔහුගේ පන්දු යැවීමේ විලාසය: "+ cricketers_si[i]["Bowling_Style"] + "."

for i in range(0,100):
    cricketers_si[i] = {key: si_schools.get(value, value) for key, value in cricketers_si[i].items()}

    

In [15]:
from google.transliteration import transliterate_text
def translate_to_sinhala(value):
	result = transliterate_text(value, lang_code='si')
	return result

In [16]:
for i in range(100):
    cricketers_si[i]["Full_Name_si"] = translate_to_sinhala(cricketers_si[i]["Full_Name"])

In [17]:
import json
with open('cricketers.json', 'w', encoding='utf-8') as f:
    json.dump(cricketers_si, f, ensure_ascii=False, indent=4)